# Run X2K

This notebook executes the eXpression2Kinases (X2K) pipeline, which takes up- and down- gene sets from each patient sample and infers the upstream TFs, intermediates and kinases likely regulating the observed differential expression. If phosphoproteomics are also available, we can find the overlap of kinases enriched in both transcriptomics and phosphoproteomics.

CSVs containing the predicted TFs, intermediates, and kinases for each sample will be produced in the specified output folder.

In [1]:
import pandas as pd
import numpy as np
import json
import os
import requests
from scipy.stats import zscore
from tqdm import tqdm

# Parameters
Customize analysis parameters below:

In [2]:
n_TFs = 20 # number of top-ranked TFs to add to the gene set
augment = 'coexpression' # method of gene set expansion ['generif', 'tagger', 'enrichr', 'coexpression']
n_genes = 100 # size of expanded gene set
rank_method = 'Integrated--meanRank' # method of kinase ranking ['Integrated--meanRank', 'Integrated--topRank'] or the name of the database
out_folder = "results" # folder to output results
rank_threshold = 30 # cut-off of top-ranked kinases considered as enriched in a sample

Functions to call the appropriate APIs:

In [8]:
## load the expected baseline KEA rank distribution for each kinase--used for rank normalization
kea_stats = pd.read_csv('data/KEA3_rank_stats.csv', index_col=0)
## load the expected baseline ChEA rank distribution for each TF--used for rank normalization
chea_stats = pd.read_csv('data/ChEA3_rank_stats.csv', index_col=0)

def get_chea3_results(gene_set, query_name='', db=rank_method):
    """ Inputs:
    gene_set: list[]
    query_name: str
    db: ['Integrated--meanRank', 'Integrated--topRank'], ranking method for ChEA3 
    Return: list[] of TFs by decreasing score (higher match) """ 

    ADDLIST_URL = 'https://maayanlab.cloud/chea3/api/enrich/'
    payload = {
        'gene_set': gene_set,
        'query_name': query_name
    }
    
    # Retrieve response
    response = requests.post(ADDLIST_URL, data=json.dumps(payload))
    if not response.ok:
        raise Exception('Error analyzing gene list')
    chea = json.loads(response.text)

    # Save nodes
    ## Lower scores indicate more relevancy to the transcription factor
    chea_results = pd.DataFrame(chea[db])
    chea_results = chea_results[['TF', 'Score']].set_index('TF')
    return chea_results

def get_kea3_results(gene_set, query_name='', db=rank_method):
    """ Inputs:
    gene_set: list[]
    query_name: str
    db: ['Integrated--meanRank', 'Integrated--topRank'], ranking method for KEA3 
    Return: list[] of kinases by decreasing score (higher match) """ 
    
    ADDLIST_URL = 'https://amp.pharm.mssm.edu/kea3/api/enrich/'
    payload = {
        'gene_set': gene_set,
        'query_name': query_name
    }
    
    # Retrieve response
    response = requests.post(ADDLIST_URL, data=json.dumps(payload))
    if not response.ok:
        raise Exception('Error analyzing gene list')
    data = json.loads(response.text)

    df = pd.DataFrame([[k['TF'], k['Score']] for k in data[db]], columns=['Kinase','Score'])
    df = df.set_index('Kinase')
    return df

def geneshot_set_augment(gene_list, similarity_type=augment, n_genes=n_genes):
    """ Inputs:
    gene_list: list[]
    similarity_type: ['generif', 'tagger', 'coexpression', 'enrichr'], resource to do the augmentation
    n_genes: int, number of the most similar genes to return
    Return: list[] of augmented genes """

    GENESHOT_URL = 'https://maayanlab.cloud/geneshot/api/associate'
    payload = {
        "gene_list": gene_list,
        "similarity": similarity_type 
        }
    
    # Retrieve response
    response = requests.post(GENESHOT_URL, json=payload)
    data = json.loads(response.text)
  
    # Return augmented genes
    geneshot_dataframe = pd.DataFrame.from_dict(data['association'], orient="index").drop(['topGenes', 'topScores'], axis=1).sort_values(by='simScore', ascending=False)
    geneshot_dataframe = geneshot_dataframe.iloc[:n_genes]
    return geneshot_dataframe.index.to_list()

def get_g2n_results(geneset, subgraph_size=30, path_length=2):
    # Inputs: 
    # 'geneset' : ['gene name'] - needs a list of strings
    # 'databases' : ['database name'] - needs a list of strings,  
    # options: iid, bioGRID, STRING, bioPlex 3.0, default - all of them 
    # 'subgraph_size' : int, - max size of the subgraph you want to produce
    # 'path_length' : int, number of edges between target proteins 

    payload = {'geneset' : geneset, 'subgraph_size': str(subgraph_size), 'path_length': path_length} 

    # Retrieve response
    res = requests.post("https://g2nkg.dev.maayanlab.cloud/api/knowledge_graph/ppi_kg", json = payload)
    results = res.json()

    # Format results
    nodes_data = {}
    for i in results:
        if i["data"]["kind"] == 'Relation':
            continue
        else:
            nodes_data[i["data"]["id"]] = i["data"]
    
    # Return nodes
    nodes = pd.DataFrame.from_dict(nodes_data, orient="index")
    return nodes['label'].to_list()

def chea_normalized(genes):
    """ Return dataframe of rank-normalized TF scores """
    
    results = get_chea3_results(gene_set=genes)
    
    # compute TF z-scores relative to the expected distribution
    z_chea = pd.DataFrame({'zscore': (chea_stats['mean'].loc[results.index]-pd.to_numeric(results['Score'])).divide(chea_stats['std'].loc[results.index])}, 
                          index=results.index) # positive value indicates smaller score (higher rank) compared to average
    return z_chea

def kea_normalized(genes):
    """ Return dataframe of rank-normalized kinase scores """
    
    results = get_kea3_results(gene_set=genes)
    
    # compute kinase z-scores relative to the expected distribution
    z_kea = pd.DataFrame({'zscore': (kea_stats['mean'].loc[results.index]-pd.to_numeric(results['Score'])).divide(kea_stats['std'].loc[results.index])}, 
                          index=results.index) # positive value indicates smaller score (higher rank) compared to average
    return z_kea

Specify the gene set files to use as input:

In [11]:
up_filename = "data/CPTAC/RNA_all.z_V1.1_isoform_adjusted_top500.tsv" # replace with your own up gene set file
dn_filename = "data/CPTAC/RNA_all.z_V1.1_isoform_adjusted_bot500.tsv" # replace with your own down gene set file

The input files should be formatted such that each column (sample ID) contains the gene set for that sample. If the TF files are already prepared, steps 1 & 2 should be skipped and the normalized TF scores can be uploaded here: 

In [ ]:
chea_up = pd.read_csv('data/CPTAC/ChEA3_10_cancer_type_meanRank_df_top500_baseline_normalized.tsv', sep='\t', index_col=0)
chea_dn = pd.read_csv('data/CPTAC/ChEA3_10_cancer_type_meanRank_df_bot500_baseline_normalized.tsv', sep='\t', index_col=0)

# 1. Up- and down- gene sets

## Load gene sets

In this example, we use a gene set size of 500 as the analysis input for each sample.

In [9]:
up_genes = pd.read_csv(up_filename, sep='\t')
dn_genes = pd.read_csv(dn_filename, sep='\t')
ids = up_genes.columns.to_list()

up_genes.head()

,C3L-00004,C3L-00010,C3L-00011,C3L-00026,C3L-00079,C3L-00088,C3L-00096,C3L-00097,C3L-00103,C3L-00183,...,17OV036,17OV039,17OV040,18OV001,20OV005,26OV002,26OV008,26OV009,26OV011,26OV013
1,OR12D1,OR52L1,ACTL8,RP11-345F18.1,INA,POU1F1,NLRP4,H3-4,OR52E8,SRMS,...,SPATA19,GAGE12J,IGHD5-12,CMTM5,HHATL,DEFA5,UCN3,DMRTC2,TRGJP,SLC26A8
2,CXCR4,OR56A1,AICDA,VGF,EFCAB3,PRDM14,CCDC105,L34079.2,OR52E6,RP11-520P18.5,...,CYP3A7,H2AB3,CD5L,GLYATL1,ALLC,DEFA6,RBL1,TM4SF20,RP11-231I13.2,CLEC1B
3,AC016577.1,OR9Q2,MRAP,TTC6,RTL1,KRT72,DPPA2,KRTAP19-1,ECT2L,HBG1,...,TRBV12-3,MAGEA6,SPO11,MYH6,C20orf202,RSPO2,PRB2,DCDC2C,WFDC9,CRNN
4,FPR3,OR56A5,DEFB135,HS3ST6,SP9,KRT73,GTSF1,NPIPA2,OR52N1,CCDC27,...,TRAV5,AHSG,FAM124B,CTXND1,CYP4F8,BSX,SERPINB12,SOHLH1,MRGPRD,CLEC2A
5,CSF2RA,MYF6,CALB2,XKR3,PRPH,KIR3DL1,SLC22A31,RPS10,GP9,TAFA3,...,CAVIN4,MAGEA3,HSD3B1,C4orf45,ACSL6,ZSCAN5B,KLHL1,TAF7L,ANGPTL5,VSTM2B


# 2. Submit up- and down- genes for TFEA

Next, the TFs predicted to regulate the up- and down- gene sets for each sample are ranked using ChEA3.

In [ ]:
# Compute normalized TF enrichment scores
chea_up = {}
chea_dn = {}

with tqdm(total=len(ids)) as pbar:
    pbar.set_description("Computing normalized ChEA scores...")
    for id in ids:
        up_results = chea_normalized(up_genes[id].to_list())
        dn_results = chea_normalized(dn_genes[id].to_list())
        if len(chea_up) == 0:
            TFs = up_results.zscore.index.to_list()
        chea_up[id] = up_results.zscore.loc[TFs]
        chea_dn[id] = dn_results.zscore.loc[TFs]
        pbar.update(1)

chea_up = pd.DataFrame(chea_up, index=TFs)
chea_dn = pd.DataFrame(chea_dn, index=TFs)
chea_up.to_csv(f"{out_folder}/up_chea_zscores.csv")
chea_dn.to_csv(f"{out_folder}/dn_chea_zscores.csv")

chea_up.head()

# 3. Gene set augmentation

In [ ]:
# Get intermediates from up- and down- TFs
up_TFs = {}
up_intermediates = {}
with tqdm(total=len(ids)) as pbar:
    pbar.set_description("Augmenting TFs from up-genes...")
    for id in ids:
        up_TFs[id] = chea_up[id].sort_values(ascending=False).index[:n_TFs].to_list()
        up_intermediates[id] = geneshot_set_augment(gene_list=up_TFs[id])
        pbar.update(1)
up_TFs = pd.DataFrame(up_TFs)
up_intermediates = pd.DataFrame(up_intermediates)
up_TFs.to_csv(f"{out_folder}/up_TFs.csv", index=False)
up_intermediates.to_csv(f"{out_folder}/up_intermediates.csv", index=False)

dn_TFs = {}
dn_intermediates = {}
with tqdm(total=len(ids)) as pbar:
    pbar.set_description("Augmenting TFs from down-genes...")
    for id in ids:
        dn_TFs[id] = chea_dn[id].sort_values(ascending=False).index[:n_TFs].to_list()
        dn_intermediates[id] = geneshot_set_augment(gene_list=dn_TFs[id])
        pbar.update(1)
dn_TFs = pd.DataFrame(dn_TFs)
dn_intermediates = pd.DataFrame(dn_intermediates)
dn_TFs.to_csv(f"{out_folder}/dn_TFs.csv", index=False)
dn_intermediates.to_csv(f"{out_folder}/dn_intermediates.csv", index=False)

In [ ]:
# Filter TFs out of intermediates
TFs = chea_up.index.to_list()
for file in os.listdir(out_folder):
    data = {}
    if file.endswith("_intermediates.csv"):
        f = pd.read_csv(f"{out_folder}/{file}")
        for id in f.columns:
            newlist = []
            for x in f[id].to_list():
                if x not in TFs: newlist.append(x)
            data[id] = newlist
        pd.DataFrame(data).to_csv(f"{out_folder}/filtered_{file}", index=False)

# 4. Kinase enrichment analysis

In [ ]:
# Compute normalized kinase enrichment scores
up_kinases = {}
dn_kinases = {}
x2k_up = {}
x2k_dn = {}

with tqdm(total=len(ids)) as pbar:
    pbar.set_description("Computing normalized KEA scores...")
    for id in ids:
        up_geneset = list(set(list(up_TFs[id]) + list(up_intermediates[id])))
        dn_geneset = list(set(list(dn_TFs[id]) + list(dn_intermediates[id])))
        up_results = kea_normalized(up_geneset)
        dn_results = kea_normalized(dn_geneset)
        up_kinases[id] = up_results.sort_values(by="zscore", ascending=False).index.to_list()
        dn_kinases[id] = dn_results.sort_values(by="zscore", ascending=False).index.to_list()
        if len(x2k_up) == 0:
            KINASES = up_results.zscore.index.to_list()
        x2k_up[id] = up_results.zscore.loc[KINASES]
        x2k_dn[id] = dn_results.zscore.loc[KINASES]
        pbar.update(1)

pd.DataFrame(up_kinases).to_csv(f"{out_folder}/up_kinases.csv", index=False)
pd.DataFrame(dn_kinases).to_csv(f"{out_folder}/dn_kinases.csv", index=False)
x2k_up = pd.DataFrame(x2k_up, index=KINASES)
x2k_dn = pd.DataFrame(x2k_dn, index=KINASES)
x2k_up.to_csv(f"{out_folder}/up_x2k_zscores.csv")
x2k_dn.to_csv(f"{out_folder}/dn_x2k_zscores.csv")

# Check kinase recovery

If phosphoproteomic data is available, we can check the agreement between the X2K kinase predictions and the direct phosphoproteomic kinase predictions. Uploade the normalized kinase scores for each sample here:

In [ ]:
kea_up = pd.read_csv('data/CPTAC/KEA3_V2_10_cancer_type_meanRank_df_top500_baseline_normalized.tsv', sep='\t', index_col=0)
kea_dn = pd.read_csv('data/CPTAC/KEA3_V2_10_cancer_type_meanRank_df_bot500_baseline_normalized.tsv', sep='\t', index_col=0)

In [6]:
ids = x2k_up.columns.intersection(kea_up.columns).to_list()

# Kinases enriched in both X2K and phospho- pipelines
rec_kinases = {}
for id in ids:
    up = x2k_up[id].sort_values(ascending=False).index.to_list()
    up_phospho = kea_up[id].sort_values(ascending=False).index.to_list()
    rec_kinases[id] = list(set(up[:rank_threshold]) & set(up_phospho[:rank_threshold]))
pd.DataFrame(rec_kinases).to_csv(f"{out_folder}/recovered_up_phospho_up_genes.csv", index=False)

rec_kinases = {}
for id in ids:
    dn = x2k_dn[id].sort_values(ascending=False).index.to_list()
    dn_phospho = kea_dn[id].sort_values(ascending=False).index.to_list()
    rec_kinases[id] = list(set(dn[:rank_threshold]) & set(dn_phospho[:rank_threshold]))
pd.DataFrame(rec_kinases).to_csv(f"{out_folder}/recovered_dn_phospho_dn_genes.csv", index=False)

rec_kinases = {}
for id in ids:
    dn = x2k_dn[id].sort_values(ascending=False).index.to_list()
    up_phospho = kea_up[id].sort_values(ascending=False).index.to_list()
    rec_kinases[id] = list(set(dn[:rank_threshold]) & set(up_phospho[:rank_threshold]))
pd.DataFrame(rec_kinases).to_csv(f"{out_folder}/recovered_up_phospho_dn_genes.csv", index=False)

rec_kinases = {}
for id in ids:
    up = x2k_up[id].sort_values(ascending=False).index.to_list()
    dn_phospho = kea_dn[id].sort_values(ascending=False).index.to_list()
    rec_kinases[id] = list(set(up[:rank_threshold]) & set(dn_phospho[:rank_threshold]))
pd.DataFrame(rec_kinases).to_csv(f"{out_folder}/recovered_dn_phospho_up_genes.csv", index=False)
